# Heart Disease Classification Using Age, Cholesterol Level and Blood Pressure

# Introduction

Heart Disease by definition is a type of disease that affects the heart. There are a couple of levels for heart disease which consist of 5 levels, but for this research, we are only looking at level 0 as someone with no heart disease, and level 1,2,3,4 which will be grouped that represent someone with a heart disease.

Blood pressure, cholesterol, and age are known as the factors for heart disease. The chance of getting heart disease increases with age. High cholesterol contributes to the development of atherosclerosis which contributes to worsening cardiovascular health. Systolic and diastolic blood pressure are indicators of cardiovascular health, and high blood pressure poses a significant risk of heart disease (American Heart Association, 2022).

In this data analysis, I would like to classify the heart disease category for an individual using age, cholesterol level and blood pressure, in order for people who are reading this project to know if they have a possibility to have a heart disease.

Question : 
Does the new observation have a heart disease by predicting it using age, cholesterol level, and resting blood pressure.

The dataset used includes data on the estimated heart disease levels of individuals from Cleveland, Hungarian, Switzerland, and Virginia. This dataset has 13 predictors and 1 target which is the heart disease level for each individual.

In this project, it will focus on the following predictors to build our classification model. Key terms of the predictors are defined below:

- age is Age of a Person (years)

- trestbps is Resting Blood Pressure (mm Hg)

- chol is Serum Cholesterol (mg/dl)

## Libraries

In [ ]:
library(repr)
library(tidyverse)
library(tidymodels)
library(reshape2)

: 

## Loading Data and Data Wrangling

In this project, I am using all of the datasets so that there would be more data to build a model with. The reason why more data is better is because by training a model with more data, there is a chance that the accuracy of the prediction will increase as well.

Process :
1. Loading the data
2. Combinining the data into one big dataset
3. Giving column names
4. Filter the variables that will be used as predictors and target
5. Remove the rows with "?" values
6. Change all of the predictors to numeric, and set the target to as factor
7. Group the target as 0 (no heart disease) and 1 (having heart disease)

**I filtered out the row with cholesterol being zero because it does not make sense if a person has no cholesterol at all.

In [ ]:
cleveland_data <- read_csv("data/heart_disease/processed.cleveland.data", col_names= FALSE, show_col_types = FALSE)
hungarian_data <- read_csv("data/heart_disease/processed.hungarian.data", col_names = FALSE, show_col_types = FALSE)
switzerland_data <- read_csv("data/heart_disease/processed.switzerland.data", col_names= FALSE, show_col_types = FALSE)
va_data <- read_csv("data/heart_disease/processed.va.data", col_names = FALSE, show_col_types = FALSE)


combined_data <- rbind(cleveland_data, hungarian_data, switzerland_data, va_data) 

colnames(combined_data) <- c("age","sex","cp","trestbps","chol","fbs","restecg","thalach","exang","oldpeak","slope","ca","thal","num")

selective_data <- combined_data |> select(age, trestbps, chol, num)

selective_data[selective_data == "?"] <- NA

clean_data <- na.omit(selective_data)

completed_data <- clean_data |>
    mutate(trestbps = as.numeric(trestbps), chol = as.numeric(chol))

heart_disease <- completed_data |>
    mutate(num = as_factor(ifelse(num == 0, 0, 1))) |>
    filter(chol > 0)

print("Table 1")
heart_disease |> head(6)

: 

## Building Model

Splitting data for our training and testing data

The reason why I split into training and testing data is to build our K-nearest neighbor classifier using the training data and to further evaluate the accuracy of our classifier using the testing data.

In [ ]:
set.seed(199)

heart_split <- initial_split(heart_disease, prop = 0.75, strata = num)
heart_train <- training(heart_split)
heart_test <- testing(heart_split)

: 

## Summarizing the Training Data

I only summarize the training data because the model is only built using the training data

In [ ]:
heart_num <- heart_train |>
    pivot_longer(cols = age:chol,
                names_to = "Variable",
                values_to = "Stat") |>
    group_by(Variable, num) |>
    summarize(Minimum = min(Stat), 
              Maximum = max(Stat), 
              Mean = mean(Stat), 
              Median = median(Stat), 
              Standard_deviation = sd(Stat))

heart_level_summary <- heart_train |>
    select(num) |>
    pivot_longer(cols = num,
                names_to = "Variable",
                values_to = "Stat") |>
    group_by(Variable) |>
    table() |>
    as.data.frame.matrix()

num_observations <- nrow(heart_train)

print("Table 2")
heart_num

print("Table 3")
heart_level_summary

: 

Firstly, the data is well balanced as the dataset has 267 0's and 239 1's as the target from table 3. By having a balanced dataset, it could help the model to predict classes better, because if it leans towards only one of the variable then the model would only predict that variable for most of the unknown data.

Secondly, the mean difference for each num is not that far from table 2. In age, the mean difference is only 5.5 units, in chol(cholesterol) it has 16.2 units of difference, and in trestbps(resting blood pressure) it has 8.4 units of difference, which would mean that the model would not predict well. For each num(class) if the variables are to close to each other, than the model will have a hard time in predicting as there is no real distinguisher between the classes.

## Visualizing Data

In [ ]:
age_dist <- ggplot(heart_train, aes(x = age, colour = num)) +
                geom_density() +
                labs(x = "Age", 
                     y = "Density", 
                     color = "Heart Disease Level", 
                     title = "Density plot of Age by Heart Disease",
                     subtitle = "Figure 1") +
                theme(text = element_text(size = 20))

age_chol_plot <- ggplot(heart_train, aes(x = age, y = chol, color = num)) +
                    geom_point() +
                    labs(x = "Age", 
                         y = "Cholesterol Level (mg/dl)", 
                         color = "Heart Disease Level", 
                         title = "Scatter plot of Cholesterol Level vs Age by Heart Disease Level",
                     subtitle = "Figure 2") +
                    theme(text = element_text(size = 20))

age_blood_plot <- ggplot(heart_train, aes(x = age, y = trestbps, color = num)) +
                    geom_point() +
                    labs(x = "Age", 
                         y = "Blood Pressure", 
                         color = "Heart Disease Level", 
                         title = "Scatter plot of Blood Pressure vs Age by Heart Disease Level",
                     subtitle = "Figure 3") +
                    theme(text = element_text(size = 20))

options(repr.plot.length = 5, repr.plot.width = 15)

: 

In [ ]:
age_dist

: 

It can be seen from figure 1 that the most amount of people that has been identified with heart disease is around 57 years old, where as the most amount of people that has no heart disease is at around 54 years old, as I have mentioned before it is quite close meaning that it is not that distinguishable and the model will have a hard time in predicting the new observation.

In [ ]:
age_chol_plot

: 

In [ ]:
age_blood_plot

: 

From figure 2 and figure 3, cholesterol vs age and blood pressure vs age, it can be seen that there are no correlation between the variables which is a good sign because it would mean that there is no multicollinearity as we cannot solve cases like that using knn. But the bad thing is that there are overlapping heart disease levels which would mean that it would be hard to predict new observations as the model would have a hard time deciding where to classify it.

## Best K

Steps to find the best k :

1. Create 5 splits for validation sets
2. Create the model and standardized recipe for tuning the classifier
3. Choose 30 values for k (from 1 - 30)
5. Create the workflow to get the accuracy for each k's

Note**

1. The reason I chose 40 as the upper limit for k is because in practice usually the best k is sqrt(n) as n is the total number of observations.
2. It can be seen from the code that the k values increments by 2 because the accuracy for the even values is the same as the k value before it.

In [ ]:
set.seed(199) 

heart_vfold <- vfold_cv(heart_train, v = 5, strata = num)

heart_spec <- nearest_neighbor(weight_fun = "rectangular", neighbors = tune()) |>
            set_engine("kknn") |>
            set_mode("classification")

heart_recipe <- recipe(num ~ ., data = heart_train) |>
                step_scale(all_predictors()) |>
                step_center(all_predictors())  

k_vals <- tibble(neighbors = seq(from = 1, to = 30, by = 2))

k_data <- workflow() |>
        add_recipe(heart_recipe) |>
        add_model(heart_spec) |>
        tune_grid(resamples = heart_vfold, grid = k_vals) |>
        collect_metrics() |>
        filter(.metric == "accuracy")

print("Table 4")
k_data

: 

To ease things up, I will be making a graph below so that it would be easier to determine which k(number of neigbours) would be the best for this model.

In [ ]:
k_plot <- ggplot(k_data, aes(x = neighbors, y = mean)) +
            geom_line() +
            geom_point() +
            labs(x = "Neighbors", 
                 y = "Mean accuracy", 
                 title = "Graph showing the mean accuracy for each neighbor level", 
                 subtitle = "Figure 4") +
            theme(text = element_text(size = 18))
k_plot

: 

In [ ]:
k_min <- k_data |>
            arrange(desc(mean)) |>
            slice(1) |>
            pull(neighbors)

k_min

: 

From figure 4, I chose k equals 21 because it has the highest accuracy while not too high of a number such that it would overfit. By the rule of thumb the optimal k should be around the value of the square root of the total observation.

## Building the model using the best k

After getting the best k, the next thing to do is :
1. Create a new model using the new k(best number of neighbours)
2. Build a new classifier using the new model and using the recipe for the workflow, and then fit our training data.

In [ ]:
heart_spec_known <- nearest_neighbor(weight_fun = "rectangular", neighbors = k_min) |>
            set_engine("kknn") |>
            set_mode("classification")

heart_fit <- workflow() |>
        add_recipe(heart_recipe) |>
        add_model(heart_spec_known) |>
        fit(heart_train)


: 

## Testing the New Classifier

Testing our new K-nearest neighbours classifier by predicting the testing data to get the accuracy of the classifier.

In [ ]:
set.seed(199)
predictions <- predict(heart_fit, heart_test) |>
                bind_cols(heart_test)

print("Table 5")
predictions |> head(6)

: 

In [ ]:
heart_metrics <- predictions |>
                metrics(truth = num, estimate = .pred_class)

heart_conf_mat <- predictions |>
                conf_mat(truth = num, estimate = .pred_class)

print("Table 6")
heart_metrics

print("Table 7")
heart_conf_mat

: 

We can see from the table 6 above that my classifier has ~57% accuracy in predicting the testing data. By looking at the information above, we can say that this classifier has mediocre accuracy in predicting heart disease based on their age, cholesterol level, and blood pressure, and should not be used for real life implementation.

## Prediction Using a New Observation

In [ ]:
set.seed(199)
new_obs <- tibble(age = runif(1,28,77), trestbps = runif(1,94,134), chol = runif(1,85,603), num = "Unknown")

obesity_new <- rbind(heart_disease, new_obs)

new_pred <- predict(heart_fit, new_obs) |>
                    bind_cols(new_obs) |>
                    rename(Prediction = .pred_class) |>
                    select(-"num")

print("Table 8")
new_pred

: 

For the new observation, my model predicts that this person does not have a heart disease.

In [ ]:
options(repr.plot.width = 16, repr.plot.length = 20)
names <- c("No Heart Disease", "Heart Disease")

matrix <- matrix(unlist(heart_conf_mat), ncol = 2, nrow = 2)

matrix_df <- round((matrix/colSums(matrix))*100,2) |>
                        as.data.frame()

colnames(matrix_df)  <- names

matrix_df_2 <-  matrix_df %>%
                mutate(Prediction = names) |>
                melt()


matrix_heatmap <- ggplot(matrix_df_2, aes(x = variable, y = Prediction, fill = value)) +
                    geom_tile() +
                    geom_text(aes(label = value), color = "white", size = 6) +
                    labs(x = "Truth", 
                         y = "Prediction", 
                         fill = "Prediction accuracy", 
                         title = "Heatmap of the confusion matrix",
                         subtitle = "Figure 5") +
                    theme(text = element_text(size = 20))
                    
matrix_heatmap

: 

Figure 5 presents a heatmap representation of a confusion matrix generated by our classifier. This matrix is a table that shows how accurate our predictions were across various classes, using shades of blue where lighter tones indicate higher accuracy. For example, when we tested our classifier on a new data point and it was classified as "no heart disease," the confusion matrix suggests there's a 60.67% chance this prediction is correct, meaning the individual likely does not have heart disease. This visualization played a pivotal role in our analysis, helping us to carefully assess how well our classifier performs in real-world scenarios.

## Discussion

### Summary

In this project, I used the K-nearest neighbors (KNN) classification model to determine the presence or absence of heart disease based on factors like age, cholesterol level, and blood pressure. Although the dataset originally contains specific levels of heart disease, my analysis focused solely on binary classification which is 0 for no heart disease and 1 for heart disease.

Applying KNN with a k-value of 21, as detailed in Figure 4, resulted in classifying a new observation as not having heart disease. When assessing the classifier's performance, it shows an approximate of 57% accuracy for this task. Figure 5's heatmap represents the confusion matrix, which shows that a newly classified observation, identified as not having heart disease, has a 60.67% probability of being correctly classified in this category.

### Impacts

Classifying individuals using a heart disease detector helps them understand their heart health based on age, blood pressure, and cholesterol levels as the predictors. When the classifier indicates heart disease, it encourages individuals to consider changes in their lifestyle habits. For example, they might choose to eat healthier foods or exercise more often, as suggested by recent studies (Gunjan, 2023), aiming to lower their cholesterol and blood pressure levels and reduce their risk of developing heart disease. These classifications raise awareness about heart disease risks and remind individuals to make better decisions that can contribute to a longer and healthier life.

### Future Questions

In the future, what can be done is dig more deeply on the levels of heart disease with a much larger dataset with more predictors(outside the given predictors in the dataset) rather than just the 3 that were used for this project such that it could give a better accuracy for the new observations. We could also look into more possible diseases that could also be related to someone having a heart disease (eg. What is the relation between heart disease and obesity?). 

## References

American Heart Association. (2022). Understanding Blood Pressure Readings. Retrieved from https://www.heart.org/en/health-topics/high-blood-pressure/understanding-blood-pressure-readings

Gunjan, A. (2023). Lifestyle changes to lower cholesterol and blood pressure levels for heart disease prevention. Retrieved from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10460604/